
# Data Processing
In this notebook we load and process the raw data to develop the final dataset for the IBM-Z Datathon. We make use of three main datasets for a list of all observed geo-effective CMEs from the post-SOHO era between 1996-2024, and three data sets for features and targets:

#### Geo-effective CMEs:
- The Richardson and Cane list; a list of near-Earth CMEs from 1996-2010.
- The George Mason University CME/ICME list; a list of geo-effective CMEs from 2007-2017.
- The NASA CME Scoreboard; a list of geo-effective CMEs from 2013-2024.

#### Features and Targets:
- The SOHO-LASCO CME Catalogue; a list of all CMEs observed from 1996-2024 containing information on physical quantities.
- OMNIWeb Plus data; a list of solar wind features associated with CMEs.
- The SIDC sunspot data; daily total sunspot number encoding information about the solar cycle from 1818-2024.



### Importing Libraries

In [1]:
import pandas as pd